# Customer Segmentation using RFM Analysis

This notebook performs RFM (Recency, Frequency, Monetary) analysis on the **Online Retail** dataset.

Change the `data_path` variable below to the location of your `Online_Retail.xlsx` file if needed.

Steps:
- Load dataset
- Clean data
- Compute RFM features
- Score and segment customers
- Visualize segments
- Provide marketing recommendations


In [ ]:
# -- 0) Setup: imports and path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import nbformat

# Path to your Excel file (already provided by you). Change if needed.
data_path = r"E:\\DATA Science\\portfolio project\\elevvo path\\Online_Retail.xlsx"

print('Data path set to:', data_path)


## 1) Load data

The UCI Online Retail dataset usually contains columns like: `InvoiceNo`, `StockCode`, `Description`, `Quantity`, `InvoiceDate`, `UnitPrice`, `CustomerID`, `Country`.


In [ ]:
df = pd.read_excel(data_path)
print('Loaded rows:', len(df))
df.head()


## 2) Quick data cleaning
- Remove rows with no `CustomerID`.
- Remove canceled transactions (InvoiceNo starting with 'C').
- Remove negative or zero quantity and unit price.
- Create `TotalPrice = Quantity * UnitPrice`.


In [ ]:
# Basic cleaning
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print('After cleaning rows:', len(df))
df.head()


## 3) Compute RFM metrics
- **Recency**: Number of days since the customer's last purchase (relative to a reference date). We'll use one day after the max invoice date as reference.
- **Frequency**: Number of invoices (orders) per customer.
- **Monetary**: Sum of spending per customer.


In [ ]:
# Reference date: one day after the last invoice in the dataset
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
reference_date

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm['Monetary'] = rfm['Monetary'].round(2)
rfm.head()


## 4) Create RFM scores (1-5)
We'll assign scores by segmenting each metric into quintiles (5 buckets). Higher score is better for Frequency and Monetary; for Recency, lower recency (more recent) is better.


In [ ]:
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5]).astype(int)

rfm['RFM_score'] = rfm['R_score'].map(str) + rfm['F_score'].map(str) + rfm['M_score'].map(str)
rfm['RFM_sum'] = rfm[['R_score','F_score','M_score']].sum(axis=1)
rfm.head()


## 5) Define segments
We'll use a simple mapping based on R and F to label common segments. You can refine these rules.


In [ ]:
def rfm_segment(df_row):
    r, f = df_row['R_score'], df_row['F_score']
    if r >= 4 and f >= 4:
        return 'Champions'
    if r >= 3 and f >= 3:
        return 'Loyal Customers'
    if r >= 4 and f <= 2:
        return 'New / Recent'
    if r <= 2 and f >= 4:
        return 'At Risk'
    if r <= 2 and f <= 2:
        return 'Lost / Inactive'
    return 'Need Attention'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)
rfm.groupby('Segment').agg({
    'CustomerID':'nunique',
    'Recency':'median',
    'Frequency':'median',
    'Monetary':['median','sum']
}).round(2)


## 6) Visualizations
Heatmap of average R, F, M per segment and bar chart of segment sizes.


In [ ]:
seg_stats = rfm.groupby('Segment').agg({'Recency':'mean','Frequency':'mean','Monetary':'mean','CustomerID':'count'}).rename(columns={'CustomerID':'Count'})
seg_stats = seg_stats.sort_values('Count', ascending=False)
seg_stats

# Heatmap (mean R, F, M)
plt.figure(figsize=(8,4))
sns.heatmap(seg_stats[['Recency','Frequency','Monetary']].T, annot=True, fmt='.1f')
plt.title('Average R, F, M by Segment')
plt.show()

# Bar chart: segment counts
plt.figure(figsize=(8,4))
seg_stats['Count'].plot(kind='bar')
plt.ylabel('Number of customers')
plt.title('Customers per Segment')
plt.show()


## 7) Simple marketing recommendations
- **Champions**: Reward with VIP perks, exclusive offers, early access.
- **Loyal Customers**: Loyalty program, cross-sell/up-sell recommendations.
- **New / Recent**: Welcome series, onboarding discounts.
- **At Risk**: Re-engagement campaigns with special offers.
- **Lost / Inactive**: Win-back campaigns, heavy discounts or surveys to understand churn.
- **Need Attention**: Personalized messaging, product recommendations.


In [ ]:
# Save results to CSV for further use
rfm.to_csv('/mnt/data/rfm_customers_segments.csv', index=False)
print('Saved rfm_customers_segments.csv to /mnt/data/')


### Notes
- If your dataset is large, consider sampling or optimizing groupby operations.
- You can refine segment rules (use clustering on RFM_sum or KMeans for data-driven segments).
- If `pd.qcut` fails due to many identical values, consider `rank()` before qcut or use custom bins.
